In [1]:
DEFAULT_SYSTEM_PROMPT = """ 
Eres un asistente especializado capaz de responder preguntas sencillas
"""

DECOMPOSITION_SYSTEM_PROMPT = """ 
Eres un experto en descomposición de problemas. Tu tarea es tomar un problema complejo y dividirlo en subproblemas más pequeños y manejables.

Tener en cuenta:
- Cada subproblema debe ser específico y manejable
- Los subproblemas deben seguir un orden lógico
- Juntos, los subproblemas deben cubrir completamente el problema original

Formato de respuesta:
- Responde ÚNICAMENTE con una lista de subproblemas
- Un subproblema por línea
- Usa viñetas (`-`)

Ejemplo de salida:
- subproblema 1
- subproblema 2
- subproblema 3
"""

SELECT_PROMPT = """ 
Eres un experto en clasificar si una entrada de usuario es un subproblema de un problema mucho mayor. 

Formato de respuesta: 
- Responde ÚNICAMENTE con TRUE o FALSE
"""

CONSTRUCT_PROMPT = """ 
Eres un experto en crear preguntas. Tu tarea es tomar una entrada del usuario y generar a partir de esta una pregunta o problema.

Instrucciones:
- Generar la pregunta sencilla, corta y clara.
"""

In [2]:
import re 
import sys 
import json
from io import StringIO
from abc import ABC, abstractmethod
from enum import Enum
from typing import Dict, List, Any, Optional
from dataclasses import dataclass

from langchain_ollama import (
  ChatOllama,
  OllamaLLM,
  OllamaEmbeddings
)
from langchain_core.messages import (
  BaseMessage,
  AIMessage,
  HumanMessage  
)
from langchain_core.prompts import (
  ChatPromptTemplate,
  MessagesPlaceholder  
)

GEMMA = "gemma3:1b"             # ollama gemma3:1b
DEEPSEEK = "deepseek-r1:1.5b"   # ollama deepseek-r1:1.5b

class LLM(ABC):
  def __init__(self, model:str="", api_key:str="", temperature:float=0.7):
    self.model = model
    self.api_key = api_key 
    self.temperature = temperature
  
  @abstractmethod
  def __call__(self, query:str) -> str:
    pass 

class Ollama(LLM):
  def __init__(self, model:str="", api_key:str = "", temperature:float = 0.7, system_prompt:str = ""):
    super().__init__(model, api_key, temperature)
    
    try:
      self.llm = ChatOllama(self.model, self.temperature)
    except Exception:
      self.llm = ChatOllama(model=GEMMA, temperature=0.7)
    
    self.system_prompt:str = system_prompt if len(system_prompt) > 0 else DEFAULT_SYSTEM_PROMPT
    self.memory:List = []
    
    self.chat_prompt = ChatPromptTemplate.from_messages(
      [
        ('system', f'{self.system_prompt}'),
        MessagesPlaceholder(variable_name='memory'),
        ('human', '{query}')
      ]
    )
    self.chain = self.chat_prompt | self.llm  
  
  def __call__(self, query:str) -> str:
    response = self.chain.invoke(
      {
        "query": HumanMessage(query),
        "memory": self.memory
      }
    )
    self.memory.append(HumanMessage(content=query))
    self.memory.append(AIMessage(content=response))
    
    return response

In [8]:
def decompose_problem(problem:str, verbose:bool=False) -> List[str]:
  """Descompone un problema complejo en una lista de subproblemas más manejables.

  Args:
    problem (str): el problema principal a descomponer

  Returns:
    List[str]: lista de subproblemas que juntos resuelven el problema original
  """
  # crear una instancia temporal para descomposición
  decomposer = ChatOllama(model=GEMMA, temperature=0.3) # temperatura más baja para mayor consistencia
  decomposition_prompt = ChatPromptTemplate.from_messages([
    ('system', DECOMPOSITION_SYSTEM_PROMPT),
    ('human', f'Descompone el siguiente problema en subproblemas:\n\n{problem}')
  ])
  chain = decomposition_prompt | decomposer
  response = chain.invoke({"problem":problem}).content
  
  if verbose: print(response)
  
  subproblems = []
  lines = response.strip().split('\n')
  for line in lines:
    line = line.strip()
    
    # limpiar viñetas y numeración
    if line.startswith('- '):
      subproblem = line[2:].strip()
    elif line.startswith('• '):
      subproblem = line[2:].strip()
    elif re.match(r'^\d+\.\s*', line):
      subproblem = re.sub(r'^\d+\.\s*', '', line).strip()
    elif line and not line.startswith('#'):
      subproblem = line.strip()
    else:
      continue
    
    if subproblem and len(subproblem) > 5:   # filtrar lineas muy cortas
      subproblems.append(subproblem)
    
  if not subproblems:
    raise Exception("ERROR: no se pudo dividir el problema")
    
  return subproblems

In [ ]:
problem = """
Descompone el problema siguiente en subproblemas:

Haz un analisis completo de cómo funciona la Inteligencia Artificial
1. Definición de qué es la inteligencia artificial
2. Aplicación
3. Ejemplos 
4. Genera un código sencillo en Python de cómo trabaja la Inteligencia Artificial
"""

subproblems = decompose_problem(problem, verbose=False)
for i, subproblem in enumerate(subproblems, 1):
  print(f"{i}. {subproblem}")

In [14]:
def select_problem(problem:str, subproblem:str, verbose:bool=False) -> bool:
  selector = ChatOllama(model=GEMMA, temperature=0.5) # temperatura más baja para mayor consistencia
  selection_prompt = ChatPromptTemplate.from_messages([
    ('system', SELECT_PROMPT),
    ('human', '{problem}')
  ])
  chain = selection_prompt | selector
  response = chain.invoke({"problem":f"Problema original:\n\n{problem}\n\nEntrada de usuario:\n\n{subproblem}"}).content
  
  if verbose: print(response)
  
  lines = response.strip().split('\n')
  if len(lines) == 1: 
    try: 
      output = True if lines[0] == "TRUE" else False if lines[0] == "FALSE" else None
      if output == None:
        return False
        raise Exception(f"ERROR al convertir a bool: {response}")
      return output
    except:
      return False
      raise Exception(f"ERROR al convertir a bool: {response}")
  return False
  raise Exception(f"ERROR al responder adecuadamente: {response}")

In [ ]:
selections = []
for subproblem in subproblems:
  if select_problem(problem, subproblem, verbose=False):
    selections.append(subproblem)

[print(f"{i}. {select}") for i, select in enumerate(selections, 1)]

In [15]:
def construct_task(subproblem:str, verbose:bool=False) -> str:
  selector = ChatOllama(model=GEMMA, temperature=0.7) # temperatura más baja para mayor consistencia
  selection_prompt = ChatPromptTemplate.from_messages([
    ('system', CONSTRUCT_PROMPT),
    ('human', 'Entrada del usuario: {problem}')
  ])
  chain = selection_prompt | selector
  response = chain.invoke({"problem":subproblem}).content
  
  if verbose: print(response)
  
  return response

In [ ]:
tasks = []
for select in selections:
  tasks.append( construct_task(select) ) 

[print(f"{i}. {task}") for i, task in enumerate(tasks, 1)]

In [16]:
problem = """
Haz un analisis completo de cómo funciona la Inteligencia Artificial
1. Definición de qué es la inteligencia artificial
2. Aplicación
3. Ejemplos 
4. Genera un código sencillo en Python de cómo trabaja la Inteligencia Artificial
"""

subproblems = decompose_problem(problem, verbose=False)
selections = [ s for s in subproblems if select_problem(problem=problem, subproblem=s) ]
tasks = [construct_task(select) for select in selections]

print(f"Cantidad de subproblemas: {len(subproblems)}")
print(f"Cantidad de subproblemas seleccionados: {len(selections)}")

[print(f"{i}. {task}") for i, task in enumerate(tasks, 1)]

Cantidad de subproblemas: 49
Cantidad de subproblemas seleccionados: 1
1. Okay, here are a few options for responding to the user's input "Predicción: Realizamos la predicción de los datos con el modelo entrenado," ranging from straightforward to slightly more engaging:

**Option 1 (Straightforward):**

"That's right!  We're using our model to make predictions."

**Option 2 (Slightly more informative):**

"Exactly! Our model is currently generating predictions based on the data you provided."

**Option 3 (A bit more engaging):**

"Yep!  We're leveraging our model to offer insights and predictions."

**Option 4 (If you want to offer a little more context):**

"Correct!  We're using our model to analyze the data and generate predictions for [briefly mention the data, e.g., sales trends, customer behavior]."


**To help me refine the response even further, could you tell me:**

*   **What kind of model is it?** (e.g., machine learning, statistical analysis, etc.)
*   **What kind of predic

[None]